In [94]:

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


from statsmodels.tsa.arima.model import ARIMA

In [1]:
!pip install scipy

In [2]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.8 MB/s eta 0:00:00m eta 0:00:010:01:01


In [93]:
from pmdarima.arima import auto_arima

In [55]:
from xgboost import XGBRegressor

In [42]:
treino = df.loc[df.index <= '2016-12-31']
validacao = df.loc[df.index > '2016-12-31']

treino.shape, validacao.shape

((384, 1), (24, 1))

In [43]:
treino.index.min(), treino.index.max()

(Timestamp('1985-01-01 00:00:00'), Timestamp('2016-12-01 00:00:00'))

In [44]:
validacao.index.min(), validacao.index.max()

(Timestamp('2017-01-01 00:00:00'), Timestamp('2018-12-01 00:00:00'))

In [45]:
treino['producao']

DATE
1985-01-01     71.5920
1985-02-01     69.7870
1985-03-01     61.6790
1985-04-01     56.7479
1985-05-01     54.6165
                ...   
2016-08-01    113.7734
2016-09-01    100.7221
2016-10-01     89.5068
2016-11-01     91.2292
2016-12-01    112.3141
Name: producao, Length: 384, dtype: float64

In [46]:
treino['producao'].shift(-1)

DATE
1985-01-01     69.7870
1985-02-01     61.6790
1985-03-01     56.7479
1985-04-01     54.6165
1985-05-01     57.3509
                ...   
2016-08-01    100.7221
2016-09-01     89.5068
2016-10-01     91.2292
2016-11-01    112.3141
2016-12-01         NaN
Name: producao, Length: 384, dtype: float64

In [47]:
treino['target'] = treino['producao'].shift(-1)
treino.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,producao,target
DATE,,
1985-01-01,71.5920,69.7870
1985-02-01,69.7870,61.6790
1985-03-01,61.6790,56.7479
1985-04-01,56.7479,54.6165
1985-05-01,54.6165,57.3509


In [48]:
treino.tail()

,producao,target
DATE,,
2016-08-01,113.7734,100.7221
2016-09-01,100.7221,89.5068
2016-10-01,89.5068,91.2292
2016-11-01,91.2292,112.3141
2016-12-01,112.3141,NaN


In [49]:
treino = treino.dropna()
treino.tail()

,producao,target
DATE,,
2016-07-01,112.4736,113.7734
2016-08-01,113.7734,100.7221
2016-09-01,100.7221,89.5068
2016-10-01,89.5068,91.2292
2016-11-01,91.2292,112.3141


In [50]:
validacao['target'] = validacao['producao'].shift(-1)
validacao.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,producao,target
DATE,,
2017-01-01,114.8282,98.2191
2017-02-01,98.2191,99.6408
2017-03-01,99.6408,85.9106
2017-04-01,85.9106,89.2053
2017-05-01,89.2053,99.1945


In [51]:
validacao.tail()

,producao,target
DATE,,
2018-08-01,113.0449,101.4058
2018-09-01,101.4058,94.4922
2018-10-01,94.4922,101.3895
2018-11-01,101.3895,110.5936
2018-12-01,110.5936,NaN


In [52]:
validacao = validacao.dropna()
validacao.tail()

,producao,target
DATE,,
2018-07-01,113.0893,113.0449
2018-08-01,113.0449,101.4058
2018-09-01,101.4058,94.4922
2018-10-01,94.4922,101.3895
2018-11-01,101.3895,110.5936


In [53]:
X_treino = treino.loc[:, ['producao']].values
y_treino = treino.loc[:, ['target']].values
X_validacao = validacao.loc[:, ['producao']].values
y_validacao = validacao.loc[:, ['target']].values

X_treino.shape, y_treino.shape, X_validacao.shape, y_validacao.shape

((383, 1), (383, 1), (23, 1), (23, 1))

In [75]:
modelo_xgba = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
modelo_xgba.fit(X_treino, y_treino)

XGBRegressor(n_estimators=1000, objective='reg:squarederror')

In [76]:
validacao.iloc[0]

producao    114.8282
target       98.2191
Name: 2017-01-01 00:00:00, dtype: float64

In [81]:
predicao = modelo_xgba.predict(X_validacao)
predicao

array([103.73107 , 103.76478 , 108.55515 ,  79.2563  ,  92.45105 ,
        92.51808 , 101.939224, 109.148705,  95.981   ,  99.392525,
        95.981   , 102.47991 , 111.25085 ,  98.55811 ,  96.829605,
        89.18475 ,  87.32575 , 100.30612 , 111.92876 , 111.92876 ,
       110.77199 ,  96.6278  , 110.77199 ], dtype=float32)

In [82]:
validacao["pred"] = predicao
validacao.head()

,producao,target,pred
DATE,,,
2017-01-01,114.8282,98.2191,103.731071
2017-02-01,98.2191,99.6408,103.764778
2017-03-01,99.6408,85.9106,108.555153
2017-04-01,85.9106,89.2053,79.256302
2017-05-01,89.2053,99.1945,92.451050


In [83]:
mean_squared_error(X_validacao, predicao)

51.31568655701552